<a href="https://colab.research.google.com/github/SHamilton94/Kickstarter-ML-/blob/master/KickstarterDataScienceML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import files
uploaded=files.upload()
import io
import matplotlib.pyplot as plt
import numpy as np
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf


Saving Kickstarter5.csv to Kickstarter5.csv


In [3]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
dataset='Kickstarter5.csv'

data = pd.read_csv(dataset, index_col=None, header=0, engine='python', na_values=[] )
data.fillna('', inplace=True)

print(data.head())
y=data.state
X=data.drop('state', axis=1)


User uploaded file "Kickstarter5.csv" with length 22774157 bytes
                                                name  ... country
0  3D Bean�� | Lifelike Fine 3D Photo Figurines o...  ...      US
1   Plug-n-Play 3D printing attachment for CNC Mills  ...      US
2  Waterless Glass Aquariums with 3D Printed Ocea...  ...      US
3                                3D Print - N - Pack  ...      US
4                                         Hobby Hand  ...      US

[5 rows x 10 columns]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)






X_train:

                                                     name  ... country
17405   Pedro 'n' Pip; Rock 'n' Roll Odyssey for Eco-K...  ...      US
38187   Smokin' Fools - Topwater Bass Fishing Live Act...  ...      US
27036                                Transgothian apparel  ...      US
36722                          PUPPYCIDE: The Documentary  ...      US
103366                              M? ?p (??t?nd?d pl??)  ...      US

[5 rows x 9 columns]
(209208, 9)

X_test:

                                                     name  ... country
68381   Playing Cards for Let Them LOL by Bicycle "The...  ...      US
121469                                 Rock on out to Sea  ...      US
49237        The Kingdoms of Caramour: The Twisted Willow  ...      US
234567  Press new David Hasselhoff on Acid album 'Euda...  ...      US
196723   Living Testimony's Debut Album " BeYond Belief "  ...      US

[5 rows x 9 columns]
(52303, 9)


In [5]:
X_train.describe()


,goal,pledged,backers
count,2.092080e+05,2.092080e+05,209208.000000
mean,3.956522e+04,1.046609e+04,122.856172
std,1.043211e+06,9.603019e+04,1059.860363
min,1.000000e-02,0.000000e+00,0.000000
25%,2.000000e+03,5.700000e+01,2.000000
50%,5.000000e+03,9.140000e+02,17.000000
75%,1.500000e+04,5.016000e+03,66.000000
max,1.000000e+08,1.328523e+07,219382.000000


In [6]:
CATEGORICAL_COLS = ['name', 'maincategory', 'currency', 'deadline', 'launched'] 
NUMERIC_COLS = ['goal', 'pledged', 'backers']
feature_columns = []
for feature_name in CATEGORICAL_COLS:
  vocab = X_train[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocab))
for feature_name in NUMERIC_COLS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float64))


In [7]:
def make_input_function(data_df, label_df, num_epochs=50, shuffle=True, batch_size=50):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds=ds.shuffle(1250)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_function(X_train, y_train)
test_input_fn = make_input_function(X_test, y_test, num_epochs=1, shuffle=False)

        


In [8]:
X_train.dtypes

name             object
maincategory     object
currency         object
deadline         object
goal            float64
launched         object
pledged         float64
backers           int64
country          object
dtype: object

In [9]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6ijxrzs1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
linear_est.train(train_input_fn)  # train


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorf

In [11]:
result = linear_est.evaluate(test_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears consoke output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model
print(result)

0.9993691
{'accuracy': 0.9993691, 'accuracy_baseline': 0.58205074, 'auc': 0.9999982, 'auc_precision_recall': 0.9999975, 'average_loss': 0.001783701, 'label/mean': 0.41794926, 'loss': 0.0017820979, 'precision': 0.99849266, 'prediction/mean': 0.41816208, 'recall': 1.0, 'global_step': 209250}


In [12]:
result = list(linear_est.predict(test_input_fn))
print(X_test.iloc[51])
print(y_test.iloc[51])
print(result[51]['probabilities'][1])

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6ijxrzs1/model.ckpt-209250
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
name            The Company Campaign. 30 Days of Love and Hustle.
maincategory                                                Music
currency                                                      USD
deadline                                